In [1]:
import sys, os
import pandas as pd
import numpy as np
sys.path.insert(0, '..')

In [13]:
from tinvest.channels import create_channel
# from tinvest.services import get_metadata
from tinvest.grpc.instruments_pb2_grpc import InstrumentsServiceStub
from tinvest.grpc.instruments_pb2 import InstrumentsRequest, InstrumentResponse, InstrumentStatus, InstrumentExchangeType

In [3]:
TOKEN = os.environ["TINVEST_READ"]

In [5]:
channel = create_channel()

In [6]:
instruments = InstrumentsServiceStub(channel)

In [18]:
request = InstrumentsRequest(
    instrument_status = InstrumentStatus.INSTRUMENT_STATUS_ALL, 
    # instrument_exchange = InstrumentExchangeType.INSTRUMENT_EXCHANGE_DEALER
)

In [14]:
def get_metadata(token: str, app_name: str = 'test-app'):
    return [("authorization", f"Bearer {token}"), ("x-app-name", app_name)]

In [15]:
metadata = get_metadata(TOKEN)

In [20]:
response = instruments.Shares(request, metadata=metadata)

In [34]:
("figi", "ticker", "class_code", "isin", "lot", "currency", "klong", "kshort", "dlong", "dshort", "dlong_min", "dshort_min", "short_enabled_flag", "name", "exchange", "ipo_date", "issue_size", "country_of_risk", "country_of_risk_name", "sector", "issue_size_plan", "nominal", "trading_status", "otc_flag", "buy_available_flag", "sell_available_flag", "div_yield_flag", "share_type", "min_price_increment", "api_trade_available_flag", "uid", "real_exchange", "position_uid", "asset_uid", "instrument_exchange", "for_iis_flag", "for_qual_investor_flag", "weekend_flag", "blocked_tca_flag", "liquidity_flag", "first_1min_candle_date", "first_1day_candle_date", "brand")

figi: "US9100471096"
ticker: "US9100471096"
class_code: "SPBXM_OTC"
isin: "US9100471096"
lot: 1
currency: "usd"
short_enabled_flag: true
name: "United Airlines Holdings"
exchange: "unknown"
issue_size: 323612861
country_of_risk: "US"
country_of_risk_name: "Соединенные Штаты Америки"
sector: "industrials"
issue_size_plan: 1000000000
nominal {
  currency: "usd"
  nano: 10000000
}
trading_status: SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING
div_yield_flag: true
share_type: SHARE_TYPE_COMMON
min_price_increment {
  nano: 10000000
}
uid: "3abf3fdb-6139-4063-a9fb-cd0483d3e95f"
real_exchange: REAL_EXCHANGE_DEALER
position_uid: "6bbde3a6-1f12-457c-a265-b19b5e9d871f"
asset_uid: "6a802865-717d-4389-823f-1bb5baaac2bf"
for_qual_investor_flag: true
blocked_tca_flag: true
brand {
  logo_name: "US9100471096.png"
  logo_base_color: "#1b4e8f"
  text_color: "#ffffff"
}

In [36]:
fields = (
    "figi", "ticker", "class_code", "isin", "lot", "currency", 
    "klong", "kshort", "dlong", "dshort", "dlong_min", "dshort_min", "short_enabled_flag", 
    "name", "exchange", "ipo_date", "issue_size", "country_of_risk", "country_of_risk_name",
    "sector", "issue_size_plan", "nominal", "trading_status", "otc_flag", "buy_available_flag",
    "sell_available_flag", "div_yield_flag", 
    "share_type", "min_price_increment", "api_trade_available_flag", "uid", 
    "real_exchange", "position_uid", "asset_uid", "instrument_exchange", 
    "for_iis_flag", "for_qual_investor_flag", "weekend_flag", "blocked_tca_flag", 
    "liquidity_flag", "first_1min_candle_date", "first_1day_candle_date", "brand"
)
for field in fields:
    print(getattr(response.instruments[0], field))

US9100471096
US9100471096
SPBXM_OTC
US9100471096
1
usd






True
United Airlines Holdings
unknown

323612861
US
Соединенные Штаты Америки
industrials
1000000000
currency: "usd"
nano: 10000000

1
False
False
False
True
1
nano: 10000000

False
3abf3fdb-6139-4063-a9fb-cd0483d3e95f
4
6bbde3a6-1f12-457c-a265-b19b5e9d871f
6a802865-717d-4389-823f-1bb5baaac2bf
0
False
True
False
True
False


logo_name: "US9100471096.png"
logo_base_color: "#1b4e8f"
text_color: "#ffffff"



In [37]:
res = []
for share in response.instruments:
    d = {}
    for field in fields:
        d[field] = getattr(share, field)
    res.append(d)
shares_df = pd.DataFrame(res)
shares_df

,figi,ticker,class_code,isin,lot,currency,klong,kshort,dlong,dshort,...,asset_uid,instrument_exchange,for_iis_flag,for_qual_investor_flag,weekend_flag,blocked_tca_flag,liquidity_flag,first_1min_candle_date,first_1day_candle_date,brand
0,US9100471096,US9100471096,SPBXM_OTC,US9100471096,1,usd,,,,,...,6a802865-717d-4389-823f-1bb5baaac2bf,0,False,True,False,True,False,,,"logo_name: ""US9100471096.png""\nlogo_base_color..."
1,TCS10A0JKQU8,RU000A0JKQU8,TILA21,RU000A0JKQU8,1,rub,,,,,...,4833e124-265f-4879-adc8-58bdf983f54e,0,False,False,False,False,True,,,"logo_name: ""RU000A0JKQU8.png""\nlogo_base_color..."
2,LU2314763264,LU2314763264,SPBXM_OTC,LU2314763264,1,usd,,,,,...,9e644a19-dc66-47e5-8251-33a0dcd7e3b0,0,False,True,False,True,True,,,"logo_name: ""LU231476326.png""\nlogo_base_color:..."
3,TCS9B6T5S470,JE00B6T5S470,00,JE00B6T5S470,1,rub,,,,,...,afbfbb30-28e4-477a-a8aa-3eb983d0d2e4,0,True,False,False,False,True,,,"logo_name: ""Solid.png""\nlogo_base_color: ""#F05..."
4,TCSF766T1007,US00766T1007,A30,US00766T1007,1,usd,,,,,...,eb89d4a2-0edb-4f3b-9e24-4ce8d2f0fe28,0,False,True,False,True,False,,,"logo_name: ""US00766T1007.png""\nlogo_base_color..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12065,BBG000FJRZ68,DOFSQ,DOFSQ,US25271C1027,1,usd,,,,,...,b7ccaca3-34f2-4ca3-ad60-29fd81507957,0,False,True,False,False,True,,,"logo_name: ""US25271C1027.png""\nlogo_base_color..."
12066,TCSF156B1070,US76156B1070,A29,US76156B1070,1,usd,,,,,...,cd3c2ab9-acb6-4dac-b8e5-f5f01488c55c,0,False,True,False,True,False,,,"logo_name: ""US76156B1070.png""\nlogo_base_color..."
12067,TCS144600507,RU0009100507,A36,RU0009100507,10,rub,,,,,...,3c04b636-d74a-4d8a-947a-1764a8c2485e,0,False,False,False,False,False,,,"logo_name: ""RU0009100507.png""\nlogo_base_color..."
12068,BBG005KPJQ48,HMHC,HMHC,US44157R1095,1,usd,,,,,...,c7df0d6d-5664-4ca0-96a2-1cb30728a003,0,False,True,False,False,True,,,"logo_name: ""US44157R1095.png""\nlogo_base_color..."


In [43]:
d['nominal']

currency: "usd"
nano: 1000000

In [39]:
shares_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12070 entries, 0 to 12069
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   figi                      12070 non-null  object
 1   ticker                    12070 non-null  object
 2   class_code                12070 non-null  object
 3   isin                      12070 non-null  object
 4   lot                       12070 non-null  int64 
 5   currency                  12070 non-null  object
 6   klong                     12070 non-null  object
 7   kshort                    12070 non-null  object
 8   dlong                     12070 non-null  object
 9   dshort                    12070 non-null  object
 10  dlong_min                 12070 non-null  object
 11  dshort_min                12070 non-null  object
 12  short_enabled_flag        12070 non-null  bool  
 13  name                      12070 non-null  object
 14  exchange              